# 0. 需要利用MEGA之类的工具生成进化关系nwk文件

# 1. 获取物种相关信息

In [2]:
from ete3 import Tree, TreeStyle, NodeStyle, AttrFace, faces, TextFace,RectFace,ImgFace
import jsonlines
from ete3 import NCBITaxa
import random


ncbi = NCBITaxa()
m = {}
tlp = "C:\\Users\\jinya\\Desktop\\bio\\env\\{}"
# info, fnwk, fpng = "pv.json", "pvf.nwk", "pvf.png"
info, fnwk, fpng = "rv.json", "rvg.nwk", "rvg.png"
# info, fnwk, fpng = "fv.json", "fve.nwk", "fev.png"


def gettax(taxid, name="species"):  # genus species
    try:
        lineage = ncbi.get_lineage(taxid)
        names = ncbi.get_taxid_translator(lineage)
        ranks = ncbi.get_rank(lineage)
        for taxid in lineage:
            if ranks[taxid] == name:
                return names[taxid]
    except:
        return


with jsonlines.open(tlp.format(info)) as fp:
    for v in fp:
        taxid = ""
        for x in v["db_xref"]:
            if "taxon:" in x:
                taxid = x[6:]
        if taxid:
            genus = gettax(taxid, "genus")
            key = "_".join(v["organism"][0].strip().split())
            m[key] = genus

def color_mapping(genus):
    # 将病毒属映射到不同的颜色
    # 这里使用一个简单的随机颜色方案，你也可以使用其他更复杂的方案
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

def get_genus_number(node):
    s = set()
    for child in node.traverse():
        if child.is_leaf():
            genus = m.get(child.name)
            s.add(genus)
    return s

# 2 绘制进化树

In [3]:
# 创建病毒属与颜色的映射关系
genus_colors = {genus: color_mapping(genus) for genus in set(m.values()) if genus}
# 读取Newick格式的进化树文件
tree = Tree(tlp.format(fnwk))
# 设置树形样式
ts = TreeStyle()
ts.show_leaf_name = False  # 显示叶子节点名称
# ts.show_branch_length = True  # 显示分支长度
# ts.show_branch_support = True  # 显示分支支持率
ts.mode = "c"  # 环状模式
ts.legend_position = 1

for node in tree.traverse():
    if node.is_leaf():
        # 设置叶子节点样式
        genus = m.get(node.name)
        nstyle = NodeStyle()
        nstyle["size"] = 0
        color = genus_colors.get(genus,"white")
        nstyle["fgcolor"] = color
        nstyle["hz_line_color"] = color  # 设置水平枝干颜色
        nstyle["vt_line_color"] = color  # 设置垂直枝干颜色
        node.set_style(nstyle)
        host = node.name[:5]
        node.name = host
        face = AttrFace("name", fsize=12)
        host_face = TextFace(f"{host}", ftype="Arial", fsize=10, fgcolor="black",penwidth=1)
        bg_face = faces.RectFace(width=100, height=20, fgcolor="black", bgcolor=color,label=host)
        host_face.background.color =color
        face.background.color =color
        node.add_face(face, column=1, position="aligned")

    else:
        # 设置内部节点样式
        nstyle = NodeStyle()
        genuses = get_genus_number(node)
        genuse=genuses.pop()
        color = "black" if len(genuses) else genus_colors.get(genuse,"black")
        nstyle["hz_line_color"] = color  # 设置水平枝干颜色
        nstyle["vt_line_color"] = color  # 设置垂直枝干颜色
        nstyle["fgcolor"] = "green"
        nstyle["size"] = 0
        node.set_style(nstyle)

    

# 添加图例
def add_legend(tree_style):
    colors = [ e  for e in genus_colors.items()]
    for i,(genus,color) in enumerate(colors):
        column = int(i/10)
        row = i-10*column
        face = TextFace(f"{genus}: ", ftype="Arial", fsize=10, fgcolor="black")
        color_face = RectFace(50, 10, fgcolor=color,bgcolor=color)
        # 使用一个虚拟节点将图例放置在树的顶部或底部
        tree_style.legend.add_face(face, column=column*2)
        tree_style.legend.add_face(color_face, column=column*2+1)


add_legend(ts)

# 绘制并显示进化树
tree.show(tree_style=ts)
# tree.render(fpng, w=256, units="mm", tree_style=ts)